# Programa auxiliar para generar transformaciones al azar en las imágenes de Carteles Letras aplicando técnicas de "Data Augmentation"
Basado y adaptado del código de https://medium.com/@bhuwanbhattarai/image-data-augmentation-and-parsing-into-an-xml-file-in-pascal-voc-format-for-object-detection-4cca3d24b33b

0) Configurar los parámetros para la ejecución:

In [0]:
# cantidad de imágenes a seleccionar por letra
#cantImSelecc = 300
cantImSelecc = 100


# directorio local en Google Drive donde están las imágenes 
path = 'gdrive/My Drive/IA/demoConvNet-Letras/Letras/'

print ("Parámetros definidos.")

1) Cargar librerías:

In [0]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

import os
import cv2
from IPython.display import Image, display
from PIL import Image as ImPIL
import numpy as np
import random
import shutil

print ("Librerías cargadas.")

2) Montar el Drive:

In [0]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

3) Definir funciones auxiliares para Data Augmentation que se usarán luego:

In [0]:

# función auxiliar para aplicar PCA color augmentation
def da_pca_color(image):

    # aplica la tansformación
    assert image.ndim == 3 and image.shape[2] == 3
    assert image.dtype == np.uint8
    img = image.reshape(-1, 3).astype(np.float32)
    sf = np.sqrt(3.0/np.sum(np.var(img, axis=0)))
    img = (img - np.mean(img, axis=0))*sf 
    cov = np.cov(img, rowvar=False) # calculate the covariance matrix
    value, p = np.linalg.eig(cov) # calculation of eigen vector and eigen value 
    rand = np.random.randn(3)*0.08
    delta = np.dot(p, rand*value)
    delta = (delta*255.0).astype(np.int32)[np.newaxis, np.newaxis, :]
    img_out = np.clip(image+delta, 0, 255).astype(np.uint8)
    
    return img_out


# función auxiliar para aplicar ruido Gaussiano
def da_noiseG(image):

    # aplica la transformación
    row,col,ch = image.shape
    mean = 0
    var = 0.01
    sigma = var**0.3
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss

    return noisy    


# función auxiliar para aplicar ruido Salt & Pepper
def da_noiseSP(image):

    # aplica la transformación
    prob = 0.05
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]

    return output    


# función auxiliar para voltear la imagen en forma vertical
def da_flip_vertical(image):

    # apica la transformación
    f_img = cv2.flip(image, 0)

    return f_img


# función auxiliar para voltear la imagen en forma horizontal
def da_flip_horizontal(image):

    # apica la transformación
    f_img = cv2.flip(image, 1)

    return f_img


# función auxiliar para rotar la imagen 90 grados
def da_rotate_90(image):

    # apica la transformación
    img_transpose = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose, 1)

    return f_img


# función auxiliar para rotar la imagen 180 grados
def da_rotate_180(image):

    # apica la transformación
    f_img = cv2.flip(image, -1)

    return f_img


# función auxiliar para rotar la imagen 270 grados
def da_rotate_270(image):

    # apica la transformación
    img_transpose_270 = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose_270, 0)

    return f_img

print ("Funciones de Data Augmentation para imágenes definidas.")

4) Procesar las imágenes por letras para generar transformaciones:

In [0]:
# levanta los xml a procesar
all_dirs = os.listdir(path)

# procesa cada directorio
for each_dir in all_dirs:
  print("\n> Procesando ", each_dir)

  allFileNames = []
  gen_FileNames = []
  noGen_FileNames = []

  dirLetras = path + '/' + each_dir

  # levanta la lista de imágenes de cada letra
  allFileNames = os.listdir(dirLetras)

  # selecciona las imágebes a procesar de la letra
  np.random.shuffle(allFileNames)
  gen_FileNames, noGen_FileNames = np.split(np.array(allFileNames),
                                                          [cantImSelecc])

  print('  -- Total: ', len(allFileNames))
  print('  -- A transformar: ', len(gen_FileNames))

  # procesa las imágenes
  cantImgGen = 0
  for img_filename in gen_FileNames:

    #print(cantImgGen, "> ", img_filename)

    # carga la imagen
    img = cv2.imread(dirLetras+'/'+img_filename)
    img_split = img_filename.strip().split('.png')

    # genera las nuevas imágenes con su correspondiente XML
    # indicar las operaciones disponibles en el vector de abajo
    #        ('c', 'ng', 'nsp', 'fv', 'fh', 'r90', 'r180', 'r270')
    # -- nota: para carteles/letras no tiene sentido voltear ni girar la imagen     
    #          el ruido gaussiano tampoco porque deforma demasiado y no se entiende
    for i in ('c', 'nsp'):

      if i=='c':        
          # aplica la transformación PCA color
          nuevaImg = da_pca_color(img)
          nuevaImgFN = img_split[0] + '-c.png'

      elif i=='ng': 
          # aplica la transformación ruido Gaussiano
          nuevaImg = da_noiseG(img)        
          nuevaImgFN = img_split[0] + '-ng.png'      

      elif i=='nsp':
          # aplica la transformación ruido Salt & Pepper
          nuevaImg = da_noiseSP(img)
          nuevaImgFN = img_split[0] + '-nsp.png'

      elif i=='fv':
          # aplica transformación de volteo vertical
          nuevaImg = da_flip_vertical(img)
          nuevaImgFN = img_split[0] + '-fv.png'

      elif i=='fh':
          # aplica transformación de volteo horizontal 
          nuevaImg = da_flip_horizontal(img)
          nuevaImgFN = img_split[0] + '-fh.png'  

      elif i=='r90':
          # aplica rotación de 90 grados 
          nuevaImg = da_rotate_90(img)
          nuevaImgFN = img_split[0] + '-r90.png'
        
      elif i=='r180':
          # aplica rotación de 180 grados 
          nuevaImg = da_rotate_180(img)
          nuevaImgFN = img_split[0] + '-r180.png'   
      
      else:
          # aplica rotación de 270 grados 
          nuevaImg = da_rotate_270(img)
          nuevaImgFN = img_split[0] + '-r270.png'      

      # si el archivo no existe
      nuevaImgFN = dirLetras + '/' + nuevaImgFN
      if not os.path.isfile(nuevaImgFN):

          # graba la nueva imagen       
          cv2.imwrite(nuevaImgFN, nuevaImg)
          
          # muestra imagen generada
          #print(nuevaImgFN)
          #display( ImPIL.fromarray(nuevaImg, 'RGB') )

          cantImgGen = cantImgGen + 1
      
  print("        Imágenes nuevas generadas: ", cantImgGen, " de ", each_dir)
